In [27]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
from torchvision.utils import save_image

import zipfile
import tqdm
import torchvision.utils as vutils

In [28]:
zip_path = os.path.join("C:/IIUM/AI Note IIUM/Deep_Learning/Week After (W9+)/data", "Anime.zip")
extract_path = "C:/IIUM/AI Note IIUM/Deep_Learning/Week After (W9+)/data/animeface"

# Only extract if not already extracted
if not os.path.exists(extract_path) or not os.listdir(extract_path):
    print("Extracting ZIP file with style...")

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        files = zip_ref.infolist()

        for file in tqdm.tqdm(files, desc="Extracting", unit="file", ncols=80, bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt}"):
            zip_ref.extract(file, extract_path)

    print("✨ Extraction complete.")
else:
    print("✅ Already extracted. Skipping extraction.")


✅ Already extracted. Skipping extraction.


## Main


In [29]:
# Generator
class Generator(nn.Module):
    def __init__(self, nz=100, ngf=64, nc=3):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()  # Output in range [-1, 1]
        )

    def forward(self, input):
        return self.main(input)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()  # Probability output
        )

    def forward(self, input):
        return self.main(input).view(-1)


In [30]:
# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256
image_size = 64
nz = 100
num_epochs = 10
lr = 0.0002
beta1 = 0.5

# Dataset loader
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Replace this with the correct path to your dataset
dataset_path = "C:/IIUM/AI Note IIUM/Deep_Learning/Week After (W9+)/data/animeface"
dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Models
netG = Generator(nz).to(device)
netD = Discriminator().to(device)

# Loss and Optimizer
criterion = nn.BCELoss()
optimizerD = torch.optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Training loop
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):
        ############################
        # (1) Update D network
        ###########################
        netD.zero_grad()
        real_images = real_images.to(device)
        b_size = real_images.size(0)
        label_real = torch.full((b_size,), 1., device=device)
        
        output = netD(real_images)
        lossD_real = criterion(output, label_real)
        lossD_real.backward()

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake_images = netG(noise)
        label_fake = torch.full((b_size,), 0., device=device)

        output = netD(fake_images.detach())
        lossD_fake = criterion(output, label_fake)
        lossD_fake.backward()

        lossD = lossD_real + lossD_fake
        optimizerD.step()

        ############################
        # (2) Update G network
        ###########################
        netG.zero_grad()
        label_g = torch.full((b_size,), 1., device=device)
        output = netD(fake_images)
        lossG = criterion(output, label_g)
        lossG.backward()
        optimizerG.step()

        if i % 50 == 0:
            print(f"[{epoch}/{num_epochs}][{i}/{len(dataloader)}] Loss_D: {lossD.item():.4f}, Loss_G: {lossG.item():.4f}")

    # ---- Save only ONCE per epoch ----
    output_dir = "output"
    model_dir = os.path.join(output_dir, "models")
    sample_dir = os.path.join(output_dir, "samples")
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(sample_dir, exist_ok=True)

    vutils.save_image(fake_images.detach(), f"{sample_dir}/epoch_{epoch:03d}.png", normalize=True)
    torch.save(netG.state_dict(), f"{model_dir}/netG_epoch_{epoch:03d}.pth")


[0/10][0/249] Loss_D: 1.6305, Loss_G: 1.9992
[0/10][50/249] Loss_D: 0.4516, Loss_G: 5.7209
[0/10][100/249] Loss_D: 1.5894, Loss_G: 2.3466
[0/10][150/249] Loss_D: 0.7059, Loss_G: 3.9036
[0/10][200/249] Loss_D: 0.5497, Loss_G: 1.3924
[1/10][0/249] Loss_D: 0.5117, Loss_G: 3.3136
[1/10][50/249] Loss_D: 0.6407, Loss_G: 5.7470
[1/10][100/249] Loss_D: 0.6042, Loss_G: 3.5425
[1/10][150/249] Loss_D: 0.3886, Loss_G: 3.8646
[1/10][200/249] Loss_D: 0.7147, Loss_G: 3.4017
[2/10][0/249] Loss_D: 0.3359, Loss_G: 4.7449
[2/10][50/249] Loss_D: 0.6946, Loss_G: 6.2319
[2/10][100/249] Loss_D: 1.0030, Loss_G: 2.3795
[2/10][150/249] Loss_D: 0.8563, Loss_G: 6.3696
[2/10][200/249] Loss_D: 0.5116, Loss_G: 4.3536
[3/10][0/249] Loss_D: 1.0649, Loss_G: 2.7839
[3/10][50/249] Loss_D: 0.7799, Loss_G: 6.7056
[3/10][100/249] Loss_D: 0.9298, Loss_G: 4.3289
[3/10][150/249] Loss_D: 1.0074, Loss_G: 8.8861
[3/10][200/249] Loss_D: 0.3409, Loss_G: 2.9857
[4/10][0/249] Loss_D: 0.2822, Loss_G: 4.7483
[4/10][50/249] Loss_D: 0.50

In [31]:
import matplotlib.pyplot as plt

def generate_images(model_path, num_images=64, nz=100, device='cuda' if torch.cuda.is_available() else 'cpu'):
    netG = Generator(nz).to(device)
    netG.load_state_dict(torch.load(model_path, map_location=device))
    netG.eval()

    with torch.no_grad():
        noise = torch.randn(num_images, nz, 1, 1, device=device)
        fake_images = netG(noise)

    # Display a grid of images
    grid = vutils.make_grid(fake_images, padding=2, normalize=True)
    plt.figure(figsize=(8,8))
    plt.axis("off")
    plt.title("Generated Images")
    plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
    plt.show()

    # Optionally save
    vutils.save_image(fake_images, "generated_output.png", normalize=True)
